# Systrace Parser

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

In [2]:
%pylab inline

import os
from time import sleep

# Support to access the remote target
import devlib
from env import TestEnv

# Import support for Android devices
from android import Screen, Workload

from devlib.utils.android import adb_command

# Support for trace events analysis
from trace import Trace

# Suport for FTrace events parsing and visualization
import trappy
import glob

import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series, DataFrame

import ipywidgets as widgets
from IPython.display import display

from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import Javascript, display

from android import System

import subprocess

from systrace_parser import systrace_parser

2019-08-24 01:22:34,410 INFO    : root         : Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt
2019-08-24 01:22:34,425 INFO    : root         : Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt


Populating the interactive namespace from numpy and matplotlib


## Test environment setup

**devlib** requires the ANDROID_HOME environment variable configured to point to your local installation of the Android SDK. If you have not this variable configured in the shell used to start the notebook server, you need to run a cell to define where your Android SDK is installed or specify the ANDROID_HOME in your target configuration.

In case more than one Android device are conencted to the host, you must specify the ID of the device you want to target in **my_target_conf**. Run **adb devices** on your host to get the ID.

In [3]:
import os
os.environ['ANDROID_HOME'] = '/home/ubuntu/Android/AndroidSDK/'

In [4]:
conf = {
    
    # Target platform and board
    "platform"    : 'android',
    "board"       : 'pixel',

    # Device ID
    #"device"      : "HT6670300102",
    
    # Android home
    "ANDROID_HOME" : "/home/ubuntu/Android/AndroidSDK",
    
    # Lisa home
    "LISA_HOME" : "/home/ubuntu/Android/lisa_old",

    # Systrace home
    "SYSTRACE_HOME" : "/ipynb/performance/testingdata",
    
    "MAX_CORE_NUM" : 8
}

## Defines

In [5]:
#file_name = 'G9750/com.eg.android.AlipayGphone_1.html'
file_name = 'com.eg.android.AlipayGphone_3.html'

## Filters

In [6]:
trace_mark_filters = {
    'launching' : {
        'SMARK' : {'context' : 'launching:', 'type' : 'S'}, 
        'EMARK' : {'context' : 'launching:', 'type' : 'F'},
    },
    'ActivityThreadMain' : {
        'SMARK' : {'context' : 'ActivityThreadMain', 'type' : 'B'}, 
        'EMARK' : {'context' : 'ActivityThreadMain', 'type' : 'E'},
    },
    'bindApplication' : {
        'SMARK' : {'context' : 'bindApplication', 'type' : 'B'}, 
        'EMARK' : {'context' : 'bindApplication', 'type' : 'E'},
    },
    'activityStart' : {
        'SMARK' : {'context' : 'activityStart', 'type' : 'B'}, 
        'EMARK' : {'context' : 'activityStart', 'type' : 'E'},
    },
    'activityResume' : {
        'SMARK' : {'context' : 'activityResume', 'type' : 'B'}, 
        'EMARK' : {'context' : 'activityResume', 'type' : 'E'},
    },
    'Choreographer' : {
        'SMARK' : {'context' : 'Choreographer#doFrame', 'type' : 'B'}, 
        'EMARK' : {'context' : 'Choreographer#doFrame', 'type' : 'E'},
        'SEPERATE' : [0, 1],
    },
}

In [ ]:
def MismatchedStringWithoutNumber(comp1, comp2):
    comp1 = unicode(comp1)
    comp2 = unicode(comp2)
    count = 0
    for index in range(min(len(comp1), len(comp2))):
        if comp1[index] != comp2[index] and not comp1[index].isnumeric() and not comp1[index].isnumeric():
            count += 1
            
    return count
#print(MismatchedStringWithoutNumber('A207FSH9_POS_190822_190822_104817_calllog?', 'A207FSH9_POS_190822_180922_104475_calllog1'))

In [74]:

count = 0
dirname = ''
dirnames = ''
chboxes = list()
chboxes_descriptions = list()

for (path, dir, files) in os.walk(conf['LISA_HOME'] + conf['SYSTRACE_HOME'] + '/'):
    if len(dir) > 0:
        dirnames = dir
    file_groups = list()
    for filename in files:
        if filename.split('.')[-1] == 'html':
            filename = filename[:-5]
            #print(filename)
            new_one = True
            for file_group in file_groups:
                #length_of_file = len(filename)
                #if length_of_file == len(file_group) and filename[:length_of_file/2] == file_group[:length_of_file/2]:
                if not MismatchedStringWithoutNumber(filename, file_group):
                    #print('before  {}'.format(file_groups))
                    #if not file_group.contains('[?]'):
                    #    file_groups.remove(file_group)
                        #file_groups.append('{}?{}'.format(file_group[:-6],file_group[-5:]))
                    #    file_groups.append('{}[?]'.format(file_group[:-3]))
                        #print('after  {}'.format(file_groups))
                    new_one = False
                    break
            if new_one:
                file_groups.append(filename)

    for file_group in file_groups:
        chboxes_descriptions.append('{}{}'.format(dirname, file_group))
    if len(dirnames) > count:
        dirname = dirnames[count] + '/'
    else:
        dirname = ''
    count += 1
    
chboxes_descriptions.sort()
#display(chboxes_descriptions)
for chboxes_description in chboxes_descriptions:
        chboxes.append(widgets.Checkbox(
                value=False,
                description=chboxes_description.decode("utf-8"),
                disabled=False,
                layout=widgets.Layout(width="200%"),
                style={'description_width': 'initial'}
        ))

toggle = widgets.ToggleButton(
    description='START'
)

#chboxes[4].value = True
#chboxes[16].value = True

display(widgets.VBox(chboxes))

VkJveChjaGlsZHJlbj0oQ2hlY2tib3godmFsdWU9RmFsc2UsIGRlc2NyaXB0aW9uPXUnQ2FsbG9nL0EyMDdGU0g5X1BPU18xOTA4MjJfMTkwODIyXzEwNDgxN19jYWxsbG8/Lmh0bWwnLCBsYXnigKY=


## Run cells

In [ ]:
def run_all(button):
    #get_ipython().run_cell()
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Run all below")
button.on_click(run_all)
display(button)

## Get file list

In [81]:
filelist = list()
title = list()

for chbox in chboxes:
    if chbox.value:
        #print(chbox.description)
        file_path = chbox.description.split('/')
        new_file_path = ''
        if len(file_path) > 1:
            for idx in range(len(file_path) - 1):
                new_file_path += file_path[idx] + '/'
        #length_of_file = len(file_path[-1])
        #new_file_path += file_path[-1][:-3] + '*.html'
        new_file_path += '*.html'
        #print(new_file_path)
        
        gathering_filelist = list()
        for dir_file in glob.glob(conf['LISA_HOME'] + conf['SYSTRACE_HOME'] + '/' + new_file_path):
            if not MismatchedStringWithoutNumber(file_path[-1], dir_file.split('/')[-1]):
                #print(dir_file)
                gathering_filelist.append(dir_file)
        filelist.append(gathering_filelist)
        #filelist[-1].sort()
        title += [file_path[-1]]
filelist

## Reading Files

In [82]:
group1 = systrace_parser(trace_mark_filters, filelist[0], title[0])
group2 = systrace_parser(trace_mark_filters, filelist[1], title[1])

In [83]:
group1.run()
group2.run()

Parsing 379575 lines are Ok
Parsing 290074 lines are Ok
Parsing 290439 lines are Ok
Parsing 274218 lines are Ok
Parsing 270095 lines are Ok
Parsing 198786 lines are Ok
Parsing 316855 lines are Ok
Parsing 299132 lines are Ok
Parsing 301678 lines are Ok
Parsing 270280 lines are Ok


## Analysis

### The timing of marking data

In [84]:
list_of_df = group1.get_marking_time()
print(group1.title)
for df in list_of_df:
    display(df * 1000)

Nytimes/A207F_Nytimes_2019_0*.html


,Choreographer #0,Choreographer #1,bindApplication,activityStart,ActivityThreadMain,activityResume,launching
pid,,,,,,,
15889.0,NaN,NaN,49.701,NaN,16.271,NaN,NaN
15977.0,NaN,NaN,721284.180,NaN,12.017,NaN,NaN
1967.0,3.433,0.130,NaN,NaN,NaN,NaN,NaN
1368.0,1.467,0.004,NaN,NaN,NaN,NaN,2148.58
15989.0,NaN,NaN,63.231,NaN,10.741,NaN,NaN
2569.0,2.426,0.052,NaN,NaN,NaN,NaN,NaN
3128.0,3.635,12.719,NaN,NaN,NaN,NaN,NaN
15709.0,429.753,69.079,618.220,680.134,14.215,320.956,NaN


,Choreographer #0,Choreographer #1,bindApplication,activityStart,ActivityThreadMain,activityResume,launching
pid,,,,,,,
16728.0,NaN,NaN,744807.077,NaN,23.548,NaN,NaN
16486.0,NaN,NaN,739081.923,NaN,13.532,NaN,NaN
16521.0,376.987,66.931,521.452,514.857,31.552,372.746,NaN
1967.0,3.062,0.123,NaN,NaN,NaN,NaN,NaN
1368.0,1.822,1.190,NaN,NaN,NaN,NaN,2012.04
16694.0,NaN,NaN,53.234,NaN,20.933,NaN,NaN
3128.0,2.557,12.128,NaN,NaN,NaN,NaN,NaN
16830.0,NaN,NaN,746107.997,NaN,10.888,NaN,NaN


,Choreographer #0,Choreographer #1,bindApplication,activityStart,ActivityThreadMain,activityResume,launching
pid,,,,,,,
3128.0,2.565,9.919,NaN,NaN,NaN,NaN,NaN
1368.0,0.017,0.005,NaN,NaN,NaN,NaN,1662.625
16915.0,321.972,58.255,512.581,424.642,12.578,266.68,NaN
17076.0,NaN,NaN,53.838,NaN,12.169,NaN,NaN
1967.0,2.910,0.125,NaN,NaN,NaN,NaN,NaN


,Choreographer #0,Choreographer #1,bindApplication,activityStart,ActivityThreadMain,activityResume,launching
pid,,,,,,,
3128.0,0.296,10.323,NaN,NaN,NaN,NaN,NaN
1368.0,2.671,1.321,NaN,NaN,NaN,NaN,1740.057
17244.0,341.337,63.117,539.758,420.782,14.315,278.948,NaN
17412.0,NaN,NaN,54.456,NaN,11.666,NaN,NaN
1967.0,3.266,0.124,NaN,NaN,NaN,NaN,NaN


,Choreographer #0,Choreographer #1,bindApplication,activityStart,ActivityThreadMain,activityResume,launching
pid,,,,,,,
3128.0,2.620,10.415,NaN,NaN,NaN,NaN,NaN
1368.0,1.304,0.020,NaN,NaN,NaN,NaN,1682.56
17759.0,NaN,NaN,49.636,NaN,11.215,NaN,NaN
17595.0,335.409,58.073,521.717,417.99,13.391,276.353,NaN
1967.0,9.921,0.116,NaN,NaN,NaN,NaN,NaN


In [85]:
list_of_df = group2.get_marking_time()
print(group2.title)
for df in list_of_df:
    display(df * 1000)

Nytimes/J810F_Nytimes_2019_0*.html


,Choreographer #0,Choreographer #1,bindApplication,activityStart,ActivityThreadMain,activityResume,launching
pid,,,,,,,
17576.0,10.380,2.575,NaN,NaN,NaN,NaN,NaN
1902.0,1.109,0.082,NaN,NaN,NaN,NaN,NaN
1680.0,5.928,0.031,NaN,NaN,NaN,NaN,NaN
3095.0,0.561,NaN,NaN,NaN,NaN,NaN,NaN
3067.0,2.385,0.133,NaN,NaN,NaN,NaN,NaN
17823.0,NaN,NaN,34.659,NaN,10.463,NaN,NaN


,Choreographer #0,Choreographer #1,bindApplication,activityStart,ActivityThreadMain,activityResume,launching
pid,,,,,,,
17957.0,309.907,80.423,540.732,431.539,9.429,312.412,NaN
1902.0,3.196,0.159,NaN,NaN,NaN,NaN,NaN
1680.0,1.542,10.380,NaN,NaN,NaN,NaN,NaN
1681.0,0.673,0.509,NaN,NaN,NaN,NaN,NaN
18130.0,NaN,NaN,27.963,NaN,8.590,NaN,NaN
1589.0,NaN,NaN,NaN,NaN,NaN,NaN,2034.003
3095.0,0.332,4.214,NaN,NaN,NaN,NaN,NaN


,Choreographer #0,Choreographer #1,bindApplication,activityStart,ActivityThreadMain,activityResume,launching
pid,,,,,,,
1902.0,1.710,0.401,NaN,NaN,NaN,NaN,NaN
1680.0,2.996,1.417,NaN,NaN,NaN,NaN,NaN
1681.0,0.593,0.619,NaN,NaN,NaN,NaN,NaN
1589.0,NaN,NaN,NaN,NaN,NaN,NaN,1998.768
3095.0,0.307,1.734,NaN,NaN,NaN,NaN,NaN
18426.0,NaN,NaN,28.147,NaN,10.258,NaN,NaN
18266.0,317.952,79.533,522.816,435.319,10.143,288.673,NaN


,Choreographer #0,Choreographer #1,bindApplication,activityStart,ActivityThreadMain,activityResume,launching
pid,,,,,,,
1902.0,3.087,0.163,NaN,NaN,NaN,NaN,NaN
1680.0,3.409,1.358,NaN,NaN,NaN,NaN,NaN
1681.0,0.956,0.540,NaN,NaN,NaN,NaN,NaN
18740.0,NaN,NaN,27.331,NaN,10.237,NaN,NaN
1589.0,NaN,NaN,NaN,NaN,NaN,NaN,2001.173
18582.0,313.043,79.783,535.037,429.343,10.624,287.631,NaN
3095.0,0.276,3.064,NaN,NaN,NaN,NaN,NaN


,Choreographer #0,Choreographer #1,bindApplication,activityStart,ActivityThreadMain,activityResume,launching
pid,,,,,,,
19072.0,NaN,NaN,29.957,NaN,9.312,NaN,NaN
1902.0,0.118,2.573,NaN,NaN,NaN,NaN,NaN
1680.0,1.344,0.036,NaN,NaN,NaN,NaN,NaN
1681.0,0.685,2.127,NaN,NaN,NaN,NaN,NaN
1589.0,NaN,NaN,NaN,NaN,NaN,NaN,2024.414
3095.0,2.746,3.437,NaN,NaN,NaN,NaN,NaN
18907.0,327.205,78.882,517.063,431.052,13.402,280.221,NaN


### Launching Time(AVERAGE)

In [107]:
list_of_df = group1.get_marking_time()
count1 = 0

for df in list_of_df:
    get_launching = df[df['launching'] > 0]

    if len(get_launching.index) :
        count1 += 1
        try :
            sum_df1 = pd.concat([sum_df1, get_launching])
        except :
            sum_df1 = get_launching

sum_df1 = DataFrame(sum_df1.sum(axis=0)).swapaxes("index", "columns") 
sum_df1.index = [group1.title]
sum_df1 = sum_df1.replace(0, NaN).dropna(axis=1)


list_of_df = group2.get_marking_time()
count2 = 0

for df in list_of_df:
    get_launching = df[df['launching'] > 0]
    
    if len(get_launching.index) :
        count2 += 1
        try :
            sum_df2 = pd.concat([sum_df2, get_launching])
        except :
            sum_df2 = get_launching
        
sum_df2 = DataFrame(sum_df2.sum()).swapaxes("index", "columns") 
sum_df2.index = [group2.title]
sum_df2 = sum_df2.replace(0, NaN).dropna(axis=1)

pd.concat([sum_df1 * 1000 / count1, sum_df2 * 1000 / count2])

Nytimes/J810F_Nytimes_2019_0*.html,Choreographer #0,Choreographer #1,bindApplication,activityStart,ActivityThreadMain,activityResume,launching
pid,,,,,,,
1368.0,1.4562,0.508,NaN,NaN,NaN,NaN,1849.1724


Nytimes/J810F_Nytimes_2019_0*.html,Choreographer #0,Choreographer #1,bindApplication,activityStart,ActivityThreadMain,activityResume,launching
pid,,,,,,,
1589.0,NaN,NaN,NaN,NaN,NaN,NaN,2014.5895


### Process Time

In [ ]:
df1 = group1.get(group1.process_state()).dropna(axis=1) * 1000
df1.columns.name = group1.title
display(df1)

df2 = group2.get(group2.process_state()).dropna(axis=1) * 1000
df2.columns.name = group2.title
display(df2)

df3 = df1 - df2
display(df3)

In [ ]:
dfs = group1.get(group1.process_state(), func='array')

for df in dfs:
    try:
        df_concat = pd.combine(df[:1], func='fill_value')
    except:
        df_concat = df[:1]
df_concat = dfs[0][:1].combine(dfs[1][:1])
display(df_concat)

### Uninterruptible Sleep

In [ ]:
index1 = group1.unint_sleep()
index2 = group2.unint_sleep()
union_index = index1
for index in index2:
    if not index in union_index:
        union_index += [index]

union_index.sort()
        
df1 = group1.get(union_index, func='avg').sort_index()
df1 = df1.dropna(axis=1)

df2 = group2.get(union_index, func='avg').sort_index()
df2 = df2.dropna(axis=1)

In [1]:
fig,axes = plt.subplots(nrows=1, ncols=2, sharey=True)
df1.plot(kind='barh', ax=axes[0], figsize=(10,8), grid=True, fontsize=12, stacked=True, title=group1.title)
df2.plot(kind='barh', ax=axes[1], figsize=(10,8), grid=True, fontsize=12, stacked=True, title=group2.title)

NameError: name 'plt' is not defined

In [ ]:
fig,axes = plt.subplots(nrows=2, ncols=1, sharey=True)
df1.sum(axis=1).plot(kind='pie', ax=axes[0], figsize=(8,16), title=group1.title)
df2.sum(axis=1).plot(kind='pie', ax=axes[1], figsize=(8,16), title=group2.title)

### Number of Running Cores

In [ ]:
df1 = group1.get(group1.cores(), func='sum')
df1 = df1.dropna(axis=1)
df1.columns.name = group1.title
#display(df1)

df2 = group2.get(group2.cores(), func='sum')
df2 = df2.dropna(axis=1)
df2.columns.name = group2.title
#display(df2)
display(pd.concat([df1, df2], keys=[group1.title, group2.title], sort=False))

plt.figure(figsize=(10, 5))
#sb.palplot(sb.color_palette("hls", 8))
subplot(1,2,1, title=group1.title)
sb.heatmap(data=df1, cmap='coolwarm', center=25, vmax = 50);
subplot(1,2,2, title=group2.title)
sb.heatmap(data=df2, cmap='coolwarm', center=25, vmax = 50);

In [ ]:
for idx in range(6):
    display(group2.parsers_of_testing[idx].get(group2.cores()))